## PARTE 1: ANÁLISE EXPLORATÓRIA DE DADOS (EDA)

### Carregamento dos Dados

Nesta etapa, realizei a leitura dos conjuntos de dados de **treino** e **teste**, que serão utilizados ao longo do projeto.

**Verificação inicial dos dados**
- Impressão do formato dos datasets (número de linhas e colunas).
- Exibição das primeiras linhas para visualizar a estrutura dos dados.


In [1]:
import os
import pandas as pd

BASE_DIR = os.getcwd()

train_path = os.path.join(BASE_DIR, "train.csv")
test_path = os.path.join(BASE_DIR, "test.csv")

df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

print("Formato dos datasets:")
print("df_train:", df_train.shape)
print("df_test :", df_test.shape)

display(df_train.head())
display(df_test.head())

Formato dos datasets:
df_train: (27579, 9)
df_test : (2753, 6)


,data,sku,cod_filial,filial,unidade,demanda,faturamento,num_transacoes,clientes_unicos
0,2024-01-02,9,101032,RUA,KG,0.430,30.12,3,3
1,2024-01-03,9,101032,RUA,KG,0.238,16.65,2,2
2,2024-01-04,9,101032,RUA,KG,0.400,27.97,2,2
3,2024-01-05,9,101032,RUA,KG,0.378,26.45,2,2
4,2024-01-06,9,101032,RUA,KG,0.766,53.58,4,4


,id,data,sku,cod_filial,filial,unidade
0,0,2024-12-02,9,101032,RUA,KG
1,1,2024-12-03,9,101032,RUA,KG
2,2,2024-12-04,9,101032,RUA,KG
3,3,2024-12-05,9,101032,RUA,KG
4,4,2024-12-06,9,101032,RUA,KG


### Valores Faltantes

Verificação da quantidade de valores ausentes (`NaN`) em cada coluna dos conjuntos de treino e teste.


In [2]:
df_train.isnull().sum()

data               0
sku                0
cod_filial         0
filial             0
unidade            0
demanda            0
faturamento        0
num_transacoes     0
clientes_unicos    0
dtype: int64

In [3]:
df_test.isnull().sum()

id            0
data          0
sku           0
cod_filial    0
filial        0
unidade       0
dtype: int64

### Dataset de Produtos

Carregamento do arquivo `produto.csv`, que contém informações descritivas dos produtos.

Nesta etapa, verifiquei o formato do dataset e visualizei as primeiras linhas para entender sua estrutura.


In [4]:
produto_path = os.path.join(BASE_DIR, "produto.csv")


df_produto = pd.read_csv(produto_path)


print("Formato dos datasets:")
print("df_produto:", df_produto.shape)

display(df_produto.head())

Formato dos datasets:
df_produto: (6500, 4)


,SKU,NOME_PRODUTO,CATEGORIA,SUBCATEGORIA
0,3.0,COCO RALADO GROSSO KG ...,Doceria,Confeitaria
1,4.0,ICE TEA LEAO LATA 340ML ...,NaN,NaN
2,5.0,TAHINE ISTAMBUL 200G ...,Pelo Mundo,Pastas Árabes
3,6.0,AMENDOIM MOIDO KG ...,Castanhas & Oleaginosas,Oleaginosas moídas
4,7.0,HALAWI ISTAMBUL LATA 500G ...,Pelo Mundo,Pastas Árabes


### Valores Ausentes no Dataset de Produtos

Verificação das colunas que possuem valores nulos no arquivo `produto.csv`, exibindo apenas aquelas com pelo menos um valor ausente.

In [5]:
nulls = df_produto.isnull().sum()
nulls[nulls > 0]

SKU                1
NOME_PRODUTO       1
CATEGORIA       2002
SUBCATEGORIA    2002
dtype: int64

## PARTE 2: TRATAMENTO E PREPARAÇÃO DOS DADOS

### Criação de Cópias dos DataFrames

Foram criadas cópias dos datasets originais (`train`, `test` e `produto`) para permitir a criação de novas features e tratamentos sem alterar os dados brutos originais.


In [6]:
df_train_features = df_train.copy()
df_test_features  = df_test.copy()
df_produto_copia = df_produto.copy()

### Tratamento de Valores Ausentes em Categorias

Valores nulos nas colunas **CATEGORIA** e **SUBCATEGORIA** foram substituídos por `"DESCONHECIDA"`, garantindo consistência nas variáveis categóricas.

In [7]:
df_produto_copia["CATEORIA"] = df_produto_copia["CATEGORIA"].fillna("DESCONHECIDA")
df_produto_copia["SUBCATEGORIA"] = df_produto_copia["SUBCATEGORIA"].fillna("DESCONHECIDA")

### Limpeza e Padronização da Coluna SKU

Linhas com **SKU nulo** foram removidas, pois o identificador é essencial para o modelo.  
Em seguida, a coluna **SKU** foi convertida para **inteiro**, garantindo padronização e evitando problemas em junções e agrupamentos.


In [8]:
df_produto_copia = df_produto_copia.dropna(subset=["SKU"])

In [9]:
df_produto_copia["SKU"] = df_produto_copia["SKU"].astype(int)

### Remoção de Colunas Não Disponíveis no Conjunto de Teste

As colunas **faturamento**, **num_transacoes** e **clientes_unicos** foram removidas do conjunto de treino, pois não existem no dataset de teste, garantindo consistência entre treino e teste.


In [10]:
df_train_features = df_train_features.drop(
    columns=["faturamento", "num_transacoes", "clientes_unicos"],
    errors="ignore"
)

### Conversão de Variáveis Categóricas em Numéricas

As colunas **filial** e **unidade** foram convertidas para valores numéricos usando mapeamento (`map`) para facilitar o uso em modelos como **CatBoost**, que aceita tanto categóricas quanto numéricas, mas pode se beneficiar da codificação.


In [11]:
map_filial = {"RUA": 0, "SHOPPING": 1}
map_unidade = {"UN": 0, "KG": 1}

In [12]:
df_train_features["filial"] = df_train_features["filial"].map(map_filial)
df_train_features["unidade"] = df_train_features["unidade"].map(map_unidade)
df_test_features["filial"] = df_test_features["filial"].map(map_filial)
df_test_features["unidade"] = df_test_features["unidade"].map(map_unidade)

###  Extração de Informação de Data

As colunas **data** foram convertidas para o tipo datetime.  
A partir delas, foi criada a feature **dia_semana**, representando o dia da semana, que pode capturar padrões sazonais de demanda.


In [13]:
df_train_features["data"] = pd.to_datetime(df_train_features["data"])
df_test_features["data"] = pd.to_datetime(df_test_features["data"])

In [14]:

df_train_features["dia_semana"] = df_train_features["data"].dt.weekday
df_test_features["dia_semana"] = df_test_features["data"].dt.weekday

### Junção com Dados do Produto

As colunas **sku** foram convertidas para inteiro para garantir compatibilidade.  
Em seguida, os datasets de treino e teste foram unidos (`merge`) com o **df_produto** usando o SKU como chave.  
Após a junção, a coluna **SKU** duplicada foi removida para evitar redundância.


In [15]:
df_train_features["sku"] = df_train_features["sku"].astype(int)
df_test_features["sku"] = df_test_features["sku"].astype(int)

In [16]:
df_train_features = df_train_features.merge(
    df_produto,
    left_on="sku",
    right_on="SKU",
    how="left"
)

In [17]:
df_test_features = df_test_features.merge(
    df_produto,
    left_on="sku",
    right_on="SKU",
    how="left"
)

In [18]:
df_train_features.drop(columns=["SKU"], inplace=True)
df_test_features.drop(columns=["SKU"], inplace=True)

### Remoção de Coluna de Nome do Produto

In [19]:
df_train_features.drop(columns=["NOME_PRODUTO"], inplace=True)
df_test_features.drop(columns=["NOME_PRODUTO"], inplace=True)

### Criação de Features de Lag e Média Móvel

1. **Ordenação por `sku` e `data`** para garantir sequência temporal correta.

2. **Lags**:
   - `lag_1`: demanda do dia anterior para cada SKU.
   - `lag_7`: demanda 7 dias atrás para cada SKU.

3. **Médias Móveis**:
   - `mm_7`: média da demanda dos últimos 7 dias (deslocada 1 dia para evitar data leakage).
   - `mm_14`: média dos últimos 14 dias (também deslocada 1 dia).

4. **Remoção de linhas com NA no treino** após cálculo das lags/médias móveis.

5. **Aplicação ao teste**:
   - Usaei a **última demanda conhecida por SKU** do treino para preencher `lag_1`, `lag_7`, `mm_7` e `mm_14` do teste.
   - Valores ainda nulos no teste são preenchidos com a **média global da demanda**.

 Resultado: todas as colunas de lag e média móvel não têm mais valores faltantes.


In [20]:
df_train_features = df_train_features.sort_values(["sku", "data"])
df_test_features  = df_test_features.sort_values(["sku", "data"])

In [21]:
df_train_features["lag_1"] = (
    df_train_features.groupby("sku")["demanda"].shift(1)
)

df_train_features["lag_7"] = (
    df_train_features.groupby("sku")["demanda"].shift(7)
)

In [22]:
df_train_features["mm_7"] = (
    df_train_features.groupby("sku")["demanda"]
    .shift(1)
    .rolling(7)
    .mean()
)

df_train_features["mm_14"] = (
    df_train_features.groupby("sku")["demanda"]
    .shift(1)
    .rolling(14)
    .mean()
)

In [23]:
df_train_features = df_train_features.dropna().reset_index(drop=True)

In [24]:
ultima_demanda_por_sku = (
    df_train_features
    .groupby("sku")["demanda"]
    .last()
)

In [25]:
df_test_features["lag_1"] = df_test_features["sku"].map(ultima_demanda_por_sku)
df_test_features["lag_7"] = df_test_features["sku"].map(ultima_demanda_por_sku)

df_test_features["mm_7"] = df_test_features["sku"].map(ultima_demanda_por_sku)
df_test_features["mm_14"] = df_test_features["sku"].map(ultima_demanda_por_sku)

In [26]:
print(df_train_features[["lag_1", "lag_7", "mm_7", "mm_14"]].isna().sum())
print(df_test_features[["lag_1", "lag_7", "mm_7", "mm_14"]].isna().sum())

lag_1    0
lag_7    0
mm_7     0
mm_14    0
dtype: int64
lag_1    12
lag_7    12
mm_7     12
mm_14    12
dtype: int64


In [27]:
media_global = df_train_features["demanda"].mean()

df_test_features[["lag_1", "lag_7", "mm_7", "mm_14"]] = (
    df_test_features[["lag_1", "lag_7", "mm_7", "mm_14"]].fillna(media_global)
)

In [28]:
print(df_test_features[["lag_1", "lag_7", "mm_7", "mm_14"]].isna().sum())

lag_1    0
lag_7    0
mm_7     0
mm_14    0
dtype: int64


### Visualização Final dos Datasets

- `df_train_features`: contém todas as features + lags/médias móveis e a **coluna target `demanda`**.  
- `df_test_features`: mesmas features de treino, mas com **`Id`** ao invés da `demanda`.  

> Todos os valores nulos foram tratados, garantindo consistência entre treino e teste.

In [29]:
display(df_train_features)

,data,sku,cod_filial,filial,unidade,demanda,dia_semana,CATEGORIA,SUBCATEGORIA,lag_1,lag_7,mm_7,mm_14
0,2024-01-11,9,101032,0,1,0.580,3,Castanhas & Oleaginosas,Castanhas de Caju,0.766,1.592,1.040857,0.823000
1,2024-01-11,9,101042,1,1,1.323,3,Castanhas & Oleaginosas,Castanhas de Caju,0.580,0.766,0.896286,0.833714
2,2024-01-12,9,101032,0,1,0.302,4,Castanhas & Oleaginosas,Castanhas de Caju,1.323,2.408,0.975857,0.909786
3,2024-01-12,9,101042,1,1,1.328,4,Castanhas & Oleaginosas,Castanhas de Caju,0.302,0.348,0.675000,0.914357
4,2024-01-13,9,101032,0,1,0.420,5,Castanhas & Oleaginosas,Castanhas de Caju,1.328,1.284,0.815000,0.915929
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26775,2024-11-28,24706,101042,1,1,1.272,3,Frutas,Secas,2.580,2.774,1.690286,1.591571
26776,2024-11-29,24706,101032,0,1,3.694,4,Frutas,Secas,1.272,0.792,1.475714,1.646000
26777,2024-11-29,24706,101042,1,1,0.508,4,Frutas,Secas,3.694,2.072,1.890286,1.748857
26778,2024-11-30,24706,101032,0,1,2.216,5,Frutas,Secas,0.508,1.330,1.666857,1.693000


In [30]:
display(df_test_features)

,id,data,sku,cod_filial,filial,unidade,dia_semana,CATEGORIA,SUBCATEGORIA,lag_1,lag_7,mm_7,mm_14
28,28,2024-12-01,9,101042,1,1,6,Castanhas & Oleaginosas,Castanhas de Caju,4.244,4.244,4.244,4.244
0,0,2024-12-02,9,101032,0,1,0,Castanhas & Oleaginosas,Castanhas de Caju,4.244,4.244,4.244,4.244
29,29,2024-12-02,9,101042,1,1,0,Castanhas & Oleaginosas,Castanhas de Caju,4.244,4.244,4.244,4.244
1,1,2024-12-03,9,101032,0,1,1,Castanhas & Oleaginosas,Castanhas de Caju,4.244,4.244,4.244,4.244
30,30,2024-12-03,9,101042,1,1,1,Castanhas & Oleaginosas,Castanhas de Caju,4.244,4.244,4.244,4.244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2750,2750,2024-12-29,24706,101042,1,1,6,Frutas,Secas,2.156,2.156,2.156,2.156
2722,2722,2024-12-30,24706,101032,0,1,0,Frutas,Secas,2.156,2.156,2.156,2.156
2751,2751,2024-12-30,24706,101042,1,1,0,Frutas,Secas,2.156,2.156,2.156,2.156
2723,2723,2024-12-31,24706,101032,0,1,1,Frutas,Secas,2.156,2.156,2.156,2.156


### Divisão Treino-Validação

1. **Separar target e features de treino**  
- `y_train = df_train_features["demanda"]` → Target.  
- `X_train = df_train_features.drop(columns=["demanda"])` → Features.

2. **Preparar features de teste**  
- `X_test = df_test_features.drop(columns=["id"])` → Remove o Id do teste.

3. **Garantir mesma estrutura de colunas**  
- `set(X_train.columns) == set(X_test.columns)` → Confere se treino e teste têm as mesmas features.  
- `X_test = X_test[X_train.columns]` → Reordena colunas para ficar igual ao treino.


In [31]:
y_train = df_train_features["demanda"]

In [32]:
X_train = df_train_features.drop(columns=["demanda"])

In [33]:
X_test = df_test_features.drop(columns=["id"])

In [34]:
print(set(X_train.columns) == set(X_test.columns))

True


In [35]:
X_test = X_test[X_train.columns]

In [36]:
!pip install catboost


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## PARTE 3: GERANDO PREVISÕES PARA SUBMISSÃO

### Treinamento CatBoost
Treinei um modelo CatBoost com 1500 iterações, `learning_rate=0.03`, `depth=5` e regularização `l2_leaf_reg=10`, usando como categóricas: `sku, cod_filial, filial, unidade, CATEGORIA, SUBCATEGORIA`. Early stopping em 200 rounds e RMSE como métrica.


In [37]:
from catboost import CatBoostRegressor

cat_features = [
    "sku",
    "cod_filial",
    "filial",
    "unidade",
    "CATEGORIA",
    "SUBCATEGORIA",
]

model = CatBoostRegressor(
    iterations=1500,
    learning_rate=0.03,
    depth=5,              # 6–7 costuma ser o sweet spot
    l2_leaf_reg=10,        # regularização (importante!)
    loss_function="RMSE",
    random_seed=42,
    early_stopping_rounds=200,
    verbose=200
)

model.fit(
    X_train,
    y_train,
    cat_features=cat_features
)

0:	learn: 7.6945614	total: 200ms	remaining: 4m 59s
200:	learn: 3.5063781	total: 10.7s	remaining: 1m 9s
400:	learn: 3.2766475	total: 23.1s	remaining: 1m 3s
600:	learn: 3.1390467	total: 35.1s	remaining: 52.5s
800:	learn: 3.0373598	total: 46.5s	remaining: 40.6s
1000:	learn: 2.9537444	total: 58.4s	remaining: 29.1s
1200:	learn: 2.8905303	total: 1m 8s	remaining: 17s
1400:	learn: 2.8345633	total: 1m 17s	remaining: 5.46s
1499:	learn: 2.8057094	total: 1m 21s	remaining: 0us


### Previsões e Ensemble
Gerei previsões (`y_pred_catboost`) e combinei com a média móvel de 7 dias (`mm_7`) usando 70% CatBoost + 30% média móvel para suavizar e melhorar o resultado final.


In [38]:
y_pred_castboost = model.predict(X_test)

In [39]:
y_pred = 0.7 * y_pred_castboost + 0.3 * df_test_features["mm_7"]

## Criação do Arquivo de Submissão

1. **Criar DataFrame:** Combina `id` do teste com as previsões finais (`y_pred`).  
2. **Verificar formato e valores nulos:** `submission.shape` e `submission.isnull().sum()`.  
3. **Garantir previsões não negativas:** Ajusta valores negativos para zero com `.clip(lower=0)`.  
4. **Salvar CSV:** `submission.to_csv("submission.csv", index=False)` para submissão.


In [40]:
submission = pd.DataFrame({
    "id": df_test_features["id"],
    "demanda": y_pred
})

In [41]:
submission.shape

(2753, 2)

In [42]:
submission.isnull().sum()

id         0
demanda    0
dtype: int64

In [43]:
submission["demanda"].min()

np.float64(0.4012455261785067)

In [44]:
submission["demanda"] = submission["demanda"].clip(lower=0)

In [45]:
submission.to_csv("submission.csv", index=False)

## Importância das Features (CatBoost)

1. **Obter importâncias:** `model.get_feature_importance()` retorna a contribuição de cada variável para o modelo.  
2. **Criar DataFrame:** Combina nomes das features (`X_train.columns`) com suas importâncias.  
3. **Ordenar:** Exibe da mais importante para a menos importante para análise e possível seleção de variáveis.  
4. **Visualizar:** `fi` mostra o ranking final de importância das features.

In [46]:
import pandas as pd

importances = model.get_feature_importance()
feature_names = X_train.columns

fi = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values(by="importance", ascending=False)

fi

,feature,importance
10,mm_7,27.636421
11,mm_14,23.198706
4,unidade,11.402915
3,filial,7.872777
6,CATEGORIA,6.102060
2,cod_filial,5.188841
0,data,4.458394
5,dia_semana,4.148261
1,sku,3.455354
8,lag_1,3.210057
